# Atividade 1



In [ ]:
# Bibliotecas utilizadas
import secrets

import gurobipy as gp
from gurobipy import *

In [ ]:
#
# Funções auxiliares
#

# Range inclusive
def inclrange(min, max):
    return range(min, max + 1)

# Inteiro aleatório entre min e max inclusive
def rand(min, max):
    return min + secrets.randbelow(max - min + 1)


### Limites das variáveis das instâncias

```
|J| = {100, 200, 300, 400, 500, 600, 700, 800, 900, 1000}
|F| ∈ [|J|, 2|J|]
|L| ∈ [5, 10]
|M| ∈ [5, 10]
|P| ∈ [5, 10]
D_j,p ∈ [10, 20]
r_m,p,l ∈ [1, 5]
R_m,f ∈ [800, 1000]
C_l,f ∈ [80, 100]
p_p,l,f ∈ [10, 100]
t_p,f,j ∈ [10, 20]
```

In [ ]:
#
# Instâncias (dados utilizados, fixos enquanto o notebook está aberto)
#
# No futuro, imprimir esses valores e substituir a variável `instancias` para fixá-los para entrega da tarefa.
#

instancias = []

for len_J in [100 * x for x in inclrange(1, 10)]:
    J = [x for x in range(0, len_J)]  # conjunto de clientes
    # Número de...
    len_F = rand(len(J), 2 * len(J))
    F = [x for x in range(0, len_F)]  # fábricas
    
    len_L = rand(5, 10)  
    L = [x for x in range(0, len_L)]  # máquinas
    
    len_M = rand(5, 10)  
    M = [x for x in range(0, len_M)]  # matérias primas
    
    len_P = rand(5, 10)
    P = [x for x in range(0, len_P)]  # produtos
    
    D = []  # D_j,p: demanda de cliente j pelo produto p
    for j in J:
        D.append([])
        for p in P:
            D[j].append([])
            D[j][p] = rand(10, 20)
    
    r = []  # r_m,p,l: quantidade de matéria prima m para produzir produto p na máquina l
    for m in M:
        r.append([])
        for p in P:
            r[m].append([])
            for l in L:
                r[m][p].append([])
                r[m][p][l] = rand(1, 5)
                
    R = []  # R_m,f: quantidade de matéria prima m disponível na fábrica f
    for m in M:
        R.append([])
        for f in F:
            R[m].append([])
            R[m][f] = rand(800, 1000)
            
    C = []  # C_l,f: capacidade de produção da máquina l na fábrica f
    for l in L:
        C.append([])
        for f in F:
            C[l].append([])
            C[l][f] = rand(80, 100)
            
    cp = []  # p_p,l,f -> cp_p,l,f: custo de produção do produto p na máquina l na fábrica f
    for p in P:
        cp.append([])
        for l in L:
            cp[p].append([])
            for f in F:
                cp[p][l].append([])
                cp[p][l][f] = rand(10, 100)
    
    ct = []  # t_p,f,j -> ct_p,f,j: custo de transporte do produto p da fábrica f ao consumidor j
    for p in P:
        ct.append([])
        for f in F:
            ct[p].append([])
            for j in J:
                ct[p][f].append([])
                ct[p][f][j] = rand(10, 20)
    
    instancia = {
        'J': J,
        'F': F,
        'L': L,
        'M': M,
        'P': P,
        'D': D,
        'r': r,
        'R': R,
        'C': C,
        'cp': cp,
        'ct': ct
    }
    instancias.append(instancia)
    print(len(instancias), end = ", ")

In [ ]:
#
# Executar solução
#

def solucionar(J, F, L, M, P, D, r, R, C, cp, ct):
    # Criação do modelo
    modelo = gp.Model("Atividade 1")
    modelo.setParam(GRB.Param.Method, 1)
    modelo.setParam(GRB.Param.Threads, 1)

    # Variáveis de decisão
    tonelada_transportada = modelo.addVars(P, F, J, lb=0, vtype=GRB.CONTINUOUS)
    tonelada_produzida = modelo.addVars(P, L, F, lb=0, vtype=GRB.CONTINUOUS)
    custo = modelo.addVar(lb=0, vtype=GRB.CONTINUOUS)
    
    # Função objetivo
    obj = quicksum(quicksum(quicksum(tonelada_produzida.sum(p,l,f) * cp[p][l][f] for p in P) for f in F) for l in L) + quicksum(quicksum(quicksum(tonelada_transportada.sum(p,f,j) * ct[p][f][j] for p in P) for f in F) for j in J)
    modelo.setObjective(obj, GRB.MINIMIZE)
    
    modelo.setParam('TimeLimit',1800)
        
    for p in P:
        for j in J:
            modelo.addConstr(quicksum(tonelada_transportada.sum(p,f,j) for f in F) == D[j][p])
            
    for p in P:
        for f in F:
            modelo.addConstr(quicksum(tonelada_transportada.sum(p,f,j) for j in J) == quicksum(tonelada_produzida.sum(p,l,f) for l in L))
    
    for f in F:
        for m in M:
            modelo.addConstr(quicksum(quicksum(tonelada_produzida.sum(p,l,f) * r[m][p][l] for p in P) for l in L) <= R[m][f])
            
    for f in F:
        for l in L:
            modelo.addConstr(quicksum(tonelada_produzida.sum(p,l,f) for p in P) <= C[l][f])
    
    modelo.update()

    modelo.write('atividade1{}clientes.lp'.format(len(J)))

    modelo.optimize()
    
    if modelo.status == GRB.Status.OPTIMAL:
        print('Solução ótima:')
        print('Custo: {}'.format(modelo.objVal))

        for p in P:
            for l in L:
                for f in F:
                    if tonelada_produzida[p,l,f].X > 0.0:
                        print('Quantidade do produto {} produzido pela máquina {} na fábrica {} : {}'.format(p,l,f,tonelada_produzida[p,l,f].X))

        for p in P:
            for f in F:
                for j in J:
                    if tonelada_transportada[p,f,j].X > 0.0:
                        print('Quantidade do produto {} transportado da fábrica {} para o cliente {} : {}'.format(p,f,j,tonelada_transportada[p,f,j].X))

        print('Solução Ótima Encontrada!')
        print('Custo: {}'.format(modelo.objVal))
    elif modelo.status == GRB.Status.TIME_LIMIT:
        print('Não foi possível encontrar uma solução ótima. Tempo excedido.')
        print('Custo: {}'.format(modelo.objVal))

        for p in P:
            for l in L:
                for f in F:
                    if tonelada_produzida[p,l,f].X > 0.0:
                        print('Quantidade do produto {} produzido pela máquina {} na fábrica {} : {}'.format(p,l,f,tonelada_produzida[p,l,f].X))

        for p in P:
            for f in F:
                for j in J:
                    if tonelada_transportada[p,f,j].X > 0.0:
                        print('Quantidade do produto {} transportado da fábrica {} para o cliente {} : {}'.format(p,f,j,tonelada_transportada[p,f,j].X))

        print('Não foi possível encontrar uma solução ótima. Tempo excedido.')
        print('Solução: ')
        print('Custo: {}'.format(modelo.objVal))
    elif modelo.status == GRB.Status.INFEASIBLE:
        print('Não foi possível encontrar uma solução ótima. Status: Inviável.')
    else:
        print('Status do modelo: {}'.format(modelo.Status))

In [ ]:
solucionar(**instancias[0])

In [ ]:
solucionar(**instancias[1])

In [ ]:
solucionar(**instancias[2])

In [ ]:
solucionar(**instancias[3])

In [ ]:
solucionar(**instancias[4])

In [ ]:
solucionar(**instancias[5])

In [ ]:
solucionar(**instancias[6])

In [ ]:
solucionar(**instancias[7])

In [ ]:
solucionar(**instancias[8])

In [ ]:
solucionar(**instancias[9])

In [ ]:
solucionar(**instancias[10])